<a href="https://colab.research.google.com/github/migostro/laboratorio-de-visao/blob/main/2_3_BoundingBoxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# utilities
from operator import truediv
import os
import time
import numpy as np
from google.colab import files, drive

# image processing
from scipy import ndimage
from skimage import io, exposure, filters, transform
from skimage import data
from skimage import color
from skimage import morphology
from skimage import segmentation, measure
from skimage.util import img_as_ubyte
from skimage.filters.rank import entropy
from sklearn.metrics import jaccard_score

# visualization
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

In [ ]:
def display_single(image):
    fig, ax = plt.subplots()
    ax = plt.imshow(image, cmap=plt.cm.gray)
    plt.show()
    plt.close()

In [ ]:
# list_filepaths(): list filepath for every filepath on a root folder
# pre-condition: (root path, empty list)
# post-condition: list with every filepath on root path
def list_filepaths(path, filepaths = []):
    for filename in os.listdir(path):
        if '.csv' not in filename and '.JPG' not in filename: # rejects csvs
            filepath = os.path.join(path, filename)
            if os.path.isfile(filepath): filepaths.append(filepath) # Adiciona apenas caminhos que são de arquivos (que no nosso caso são imagens)
            else: list_filepaths(filepath, filepaths)

    return filepaths

In [ ]:
def crop_image(img, bbox): 
    return img[bbox[0]:bbox[2], bbox[1]:bbox[3]]

In [ ]:
# 0 FILE READING
drive.mount('/content/drive')
root = '/content/drive/MyDrive/Colab/MAC0417/Trabalho/0_new'
truth_path = root + "/groundTruth_resized"
segmented_path = root + "/segmented_resized"
bbox_path = root + "/bbox_resized"
bbox_truth_path = root + "/bbox_truth_resized"

segmentedData_filepaths = list_filepaths(segmented_path, [])
truthData_filepaths = list_filepaths(truth_path, [])
print(f'# of images: {len(segmentedData_filepaths)}')
print(f'# of images: {len(truthData_filepaths)}')

NameError: ignored

In [ ]:
for idx, img_path in enumerate(truthData_filepaths):
    print(idx, img_path)
    img = io.imread(img_path)[:,:,0]
    filename = img_path.split('/')[-1]
    middle_path = img_path.replace(truth_path, '').replace(filename, '')

    label_img = measure.label(img)
    regions = measure.regionprops(label_img)

    if(len(regions) > 1): # isolate_largest_area
        largest_region = max(regions, key=lambda region: region.area)
        img = crop_image(img, largest_region.bbox)

        if not os.path.isdir(bbox_truth_path + middle_path): os.makedirs(bbox_truth_path + middle_path)
        io.imsave(img_path.replace(truth_path, bbox_truth_path), img)

    if len(regions) == 1:
        img = crop_image(img, regions[0].bbox)

        if not os.path.isdir(bbox_truth_path + middle_path): os.makedirs(bbox_truth_path + middle_path)
        io.imsave(img_path.replace(truth_path, bbox_truth_path), img)

In [ ]:
classes = np.array(['borrachas', 'cartas', 'celulares', 'conchas', 'copos', 'dados', 'estatuas', 'lapis', 'tubos', 'vasos'])
scores =	{
  'borrachas': [],
  'cartas': [],
  'celulares': [],
  'conchas': [],
  'copos': [],
  'dados': [],
  'estatuas': [],
  'lapis': [],
  'tubos': [],
  'vasos': [],
}

for idx, img_path in enumerate(segmentedData_filepaths):
    print(idx, img_path)
    filename = img_path.split('/')[-1]
    middle_path = img_path.replace(segmented_path, '').replace(filename, '')

    # read
    img = io.imread(img_path)
    hasTruth = False
    if os.path.isfile(img_path.replace(segmented_path, truth_path).replace('.JPG', '.jpg')):
        truth = io.imread(img_path.replace(segmented_path, truth_path).replace('.JPG', '.jpg'))
        truth = truth > filters.threshold_yen(truth)
        hasTruth = True

    # measure
    label_img = measure.label(img)
    regions = measure.regionprops(label_img)

    if(len(regions) > 1): # isolate_largest_area
        largest_region = max(regions, key=lambda region: region.area)
        img = crop_image(img, largest_region.bbox) # (min_row, min_col, max_row, max_col)
        
        if not os.path.isdir(bbox_path + middle_path): os.makedirs(bbox_path + middle_path)
        io.imsave(img_path.replace(segmented_path, bbox_path), img)

        if hasTruth:
            truth_label_img = measure.label(img)
            truth_regions = measure.regionprops(truth_label_img)

            xA = max(truth_regions[0].bbox[0], largest_region.bbox[0])
            yA = max(truth_regions[0].bbox[1], largest_region.bbox[1])
            xB = min(truth_regions[0].bbox[2], largest_region.bbox[2])
            yB = min(truth_regions[0].bbox[3], largest_region.bbox[3])
            intersectArea = (xB - xA + 1) * (yB - yA + 1)
            unionArea = (truth_regions[0].area + largest_region.area) - intersectArea
            score = np.abs(intersectArea/unionArea)

    if len(regions) == 1:
        img = crop_image(img, regions[0].bbox)

        if not os.path.isdir(bbox_path + middle_path): os.makedirs(bbox_path + middle_path)
        io.imsave(img_path.replace(segmented_path, bbox_path), img)

        if hasTruth:
            truth_label_img = measure.label(img)
            truth_regions = measure.regionprops(truth_label_img)

            xA = max(truth_regions[0].bbox[0], regions[0].bbox[0])
            yA = max(truth_regions[0].bbox[1], regions[0].bbox[1])
            xB = min(truth_regions[0].bbox[2], regions[0].bbox[2])
            yB = min(truth_regions[0].bbox[3], regions[0].bbox[3])
            intersectArea = (xB - xA + 1) * (yB - yA + 1)
            unionArea = (truth_regions[0].area + regions[0].area) - intersectArea
            score = np.abs(intersectArea/unionArea)

    if hasTruth:
        for i in range(len(classes)): # encontra de qual classe é a imagem
            if classes[i] == img_path.split('/')[-3]:
                scores[classes[i]].append(score)

In [ ]:
# get average jaccard scores
print('[1] Average Jaccard Scores')
print(scores['conchas'])
for key, value in scores.items():
    if value: print(key, np.average(value), np.median(value), len(value))

[1] Average Jaccard Scores
[1.0463412556435812, 1.0868024951420958, 1.0857752890738417, 1.1071788880298707, 0.717669978570037, 18.08108108108108, 6.941001600731763, 3.017646106524203, 2.979064135614405, 17.15210355987055]
borrachas 0.6508292741062833 0.4059015717300185 16
cartas 0.6002764325471772 0.07068657838325901 16
celulares 1.144038094029533 0.5309809224608419 11
conchas 5.321466439028143 2.0431215118221377 10
copos 0.44176799887934354 0.1329416531604538 15
dados 1.2361908044812664 1.026005631828598 16
estatuas 1.5057597157380531 1.0645411980131998 16
lapis 0.7074368783305363 0.7249209265803823 16
tubos 1.4206435610381634 1.4955068018450444 15
vasos 1.2008590262741121 0.8470648861616804 16


In [ ]:
for idx, img_path in enumerate(segmentedData_filepaths):
    print(idx, img_path)
    img = io.imread(img_path)
    filename = img_path.split('/')[-1]
    middle_path = img_path.replace(segmented_path, '').replace(filename, '')

    label_img = measure.label(img)
    regions = measure.regionprops(label_img)

    if(len(regions) > 1): # isolate_largest_area
        largest_region = max(regions, key=lambda region: region.area)
        img = crop_image(img, largest_region.bbox)

        if not os.path.isdir(bbox_path + middle_path): os.makedirs(bbox_path + middle_path)
        io.imsave(img_path.replace(segmented_path, bbox_path), img)

    if len(regions) == 1:
        img = crop_image(img, regions[0].bbox)

        if not os.path.isdir(bbox_path + middle_path): os.makedirs(bbox_path + middle_path)
        io.imsave(img_path.replace(segmented_path, bbox_path), img)